In [85]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
import json
import numpy as np

In [129]:
load_dotenv()
opensea_api_key = os.getenv('OPENSEA_API_KEY')

# Data for Context and Analysis

### Collection Stats

In [4]:
url = "https://api.opensea.io/api/v2/collections/pudgypenguins/stats"

headers = {
    "accept": "application/json",
    "x-api-key": opensea_api_key
}

response = requests.get(url, headers=headers)

collection_stats_response_json = response.json()

collection_totals_dict = collection_stats_response_json["total"]

# I had to set this index because there is not index and it was causing an error
# I used ChatGPT to debug this issue
collection_totals_df = pd.DataFrame(collection_totals_dict, index=[0])

collection_totals_df

,volume,sales,average_price,num_owners,market_cap,floor_price,floor_price_symbol
0,343690.47517,78770,4.363215,4763,92617.799483,13.175,ETH


## Trait Counts

In [5]:
url = "https://api.opensea.io/api/v2/traits/pudgypenguins"

headers = {
    "accept": "application/json",
    "x-api-key": opensea_api_key
}

response = requests.get(url, headers=headers)

# Formatting as json
traits_json = response.json()

# get the "counts" and not "categories" key
traits_json = traits_json.get("counts", [])

# preview
traits_json

{'Background': {'Beige': 1152,
  'Blue': 1587,
  'Mint': 1389,
  'Red': 755,
  'Tangerine': 1135,
  'Yellow': 873,
  'Pink': 710,
  'Purple': 1282,
  'On The Beach': 1,
  'Supermarket': 1,
  'Green': 1,
  'Trick Or Treating': 1,
  'Underwater': 1},
 'Skin': {'Light Gray': 1189,
  'Mint': 423,
  'Maroon': 731,
  'Normal': 2649,
  'Leopard Pink': 89,
  'Dark Gray': 1329,
  'Baby Pink': 623,
  'Cream': 646,
  'Olive Green': 705,
  'Red': 344,
  'Gold': 44,
  'Leopard Gray': 89,
  'Ice': 22,
  'Navy Blue': 4,
  'Black': 1},
 'Body': {'Vote 4 Pudgy': 203,
  'Kimono Blue': 108,
  'Scarf Blue': 237,
  'Shirt Blue': 290,
  'Fish Lover': 70,
  'Turtleneck Grey': 210,
  'Lei Pink': 168,
  'Bow Tie Blue': 154,
  'Tank Top Yellow': 193,
  'Puffer Orange': 158,
  'Huddle Shirt': 228,
  'Kimono Brown': 268,
  'Lei Assorted': 76,
  'Bronze Medal': 193,
  'Gold Medal': 58,
  'Heart': 102,
  'Turtleneck Green': 254,
  'Scarf Pink': 177,
  'Surfboard Necklace': 166,
  'Swordman': 62,
  'Christmas Sweate

#### All Traits

In [6]:
# df of all traits
# documentation: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html
# similar to how OneHotEncoder creates new column names
traits_df_normalized = pd.json_normalize(traits_json)

traits_df_normalized

,Background.Beige,Background.Blue,Background.Mint,Background.Red,Background.Tangerine,Background.Yellow,Background.Pink,Background.Purple,Background.On The Beach,Background.Supermarket,...,Head.Sideways Red,Head.Egg Gold,Head.Flower Crown,Head.Fish Orange,Head.Ice Crown,Head.Hatched Gold,Head.Pineapple Suit,Head.Banana Suit,Head.Normal,Head.Shark Suit
0,1152,1587,1389,755,1135,873,710,1282,1,1,...,90,45,66,44,14,9,1,1,1,1


#### Isolating Traits

In [7]:
# show all trait keys
traits_json.keys()

dict_keys(['Background', 'Skin', 'Body', 'Face', 'Head'])

In [8]:
# explode the df
traits_df_exploded =  pd.DataFrame(traits_json)
traits_df_exploded = traits_df_exploded.explode("Background")

# create new dfs for each trait, sorted by values
# if you don't do reset_index(drop=True), it appears as a series
background_trait_df = traits_df_exploded["Background"].dropna().sort_values(ascending=False).reset_index()
skin_trait_df = traits_df_exploded["Skin"].dropna().sort_values(ascending=False).reset_index()
body_trait_df = traits_df_exploded["Body"].dropna().sort_values(ascending=False).reset_index()
face_trait_df = traits_df_exploded["Face"].dropna().sort_values(ascending=False).reset_index()
head_trait_df = traits_df_exploded["Head"].dropna().sort_values(ascending=False).reset_index()

# rename columns and set indices
background_trait_df = background_trait_df.rename(columns={"index":"background", "Background":"count"}).set_index("background")
skin_trait_df = skin_trait_df.rename(columns={"index":"skin", "Skin":"count"}).set_index("skin")
body_trait_df = body_trait_df.rename(columns={"index":"body", "Body":"count"}).set_index("body")
face_trait_df = face_trait_df.rename(columns={"index":"face", "Face":"count"}).set_index("face")
head_trait_df = head_trait_df.rename(columns={"index":"head", "Head":"count"}).set_index("head")

# display trait dataframes
display(background_trait_df, skin_trait_df, body_trait_df, face_trait_df, head_trait_df)

,count
background,
Blue,1587.0
Mint,1389.0
Purple,1282.0
Beige,1152.0
Tangerine,1135.0
Yellow,873.0
Red,755.0
Pink,710.0
On The Beach,1.0


,count
skin,
Normal,2649.0
Dark Gray,1329.0
Light Gray,1189.0
Maroon,731.0
Olive Green,705.0
Cream,646.0
Baby Pink,623.0
Mint,423.0
Red,344.0


,count
body,
Shirt Blue,290.0
Kimono Brown,268.0
Shirt Red,259.0
Turtleneck Green,254.0
Scarf Green,250.0
...,...
Pineapple Suit,1.0
Banana Suit,1.0
Mirrored,1.0


,count
face,
Winking,961.0
Blushing,932.0
Normal,864.0
Circle Glasses,768.0
Cute,588.0
Monocle,496.0
Squad,445.0
Eyepatch,426.0
Cross Eyed,417.0


,count
head,
Headband,357.0
Flat Cap Blue,355.0
Bucket Hat Green,349.0
Backwards Hat Red,294.0
Hat Blue,286.0
Top Hat,282.0
Hat Red,279.0
Beanie Orange,276.0
Flat Cap Tan,274.0


# Data for Model

## NFTs

In [9]:
url = "https://api.opensea.io/api/v2/collection/pudgypenguins/nfts"

headers = {
    "accept": "application/json",
    "x-api-key": opensea_api_key
}

response = requests.get(url, headers=headers)

# Formatting as json
nfts_json = response.json()

nfts_df = pd.DataFrame(nfts_json["nfts"])

# rename "name" to "asset.name" to combine dfs
nfts_df.rename(columns={"name":"asset.name"}, inplace=True)

display(nfts_df.sample(3))

display(nfts_df.columns)

display(len(nfts_df))

,identifier,collection,contract,token_standard,asset.name,description,image_url,metadata_url,opensea_url,updated_at,is_disabled,is_nsfw
28,8828,pudgypenguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,erc721,Pudgy Penguin #8828,A collection 8888 Cute Chubby Pudgy Penquins s...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2023-07-26T20:09:57.859296,False,False
40,8859,pudgypenguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,erc721,Pudgy Penguin #8859,A collection 8888 Cute Chubby Pudgy Penquins s...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-01T00:22:02.321751,False,False
14,8876,pudgypenguins,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,erc721,Pudgy Penguin #8876,A collection 8888 Cute Chubby Pudgy Penquins s...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2023-07-26T20:11:40.224767,False,False


Index(['identifier', 'collection', 'contract', 'token_standard', 'asset.name',
       'description', 'image_url', 'metadata_url', 'opensea_url', 'updated_at',
       'is_disabled', 'is_nsfw'],
      dtype='object')

50

## Sales Sample (50) "Events"

In [27]:

url = "https://api.opensea.io/api/v2/events/collection/pudgypenguins?event_type=sale"

headers = {
    "accept": "application/json",
    "x-api-key": opensea_api_key
}

response = requests.get(url, headers=headers)

# Formatting as json
events_json = response.json()

# get the "asset events" key and all p
events_json = events_json.get("asset_events", [])

# create df
events_df = pd.json_normalize(events_json)

# rename nft.name to asset.name
events_df.rename(columns={"nft.name":"asset.name"}, inplace=True)

display(events_df.head())

display(events_df.columns)

display(len(events_df))

,event_type,order_hash,chain,protocol_address,closing_date,quantity,seller,buyer,transaction,event_timestamp,...,nft.image_url,nft.metadata_url,nft.opensea_url,nft.updated_at,nft.is_disabled,nft.is_nsfw,payment.quantity,payment.token_address,payment.decimals,payment.symbol
0,sale,0x2f423051b75ade3673b75fba44a1c934fae6d27f816d...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1714075583,1,0x2ebb3cf9d35e5a69419b9c9a7764f2fa4fe2b930,0xcae2ff6837ec6c8690675b5f1e619c04a77f3114,0x0eab7f5e06c9214fb782f39a04bd55fd564e170b1b94...,1714075583,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-25T20:06:25.873055,False,False,13690000000000000000,0x0000000000000000000000000000000000000000,18,ETH
1,sale,,ethereum,,1714073603,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x218e312ff5181290a46e3f87a73a8ad40c05a944,0x9a6b3aef1920c07cd61da8ad7895a5f4932e33df5cb3...,1714073603,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-25T19:33:25.760992,False,False,12250000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH
2,sale,0x041251e52c4dbd0a4b55347649fd2e60f56572415e07...,ethereum,0x0000000000000068f116a894984e2db1123eb395,1714067747,1,0x24f1fce2f3742fb9895fe672994fbec949241f1d,0xac32967007909f5b312557e4909cb73a3dea3d8d,0xff5de2a81422c58530c593d7f7f2e5b61fd0ca3210ea...,1714067747,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-25T17:55:49.350564,False,False,12960000000000000000,0x0000000000000000000000000000000000000000,18,ETH
3,sale,,ethereum,,1714067723,1,0xac32967007909f5b312557e4909cb73a3dea3d8d,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x3f1784b1e73138ca38a04955bb6d6f7be2aec7fa49ab...,1714067723,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-25T17:55:26.298519,False,False,12361800000000000000,0x0000000000000000000000000000000000000000,18,ETH
4,sale,,ethereum,,1714058999,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x13747adca0ce7e7f6aa661718a07e4c74e62ed19,0x4d549d05db59caa63f69bd2171ad88a3b68d119d11af...,1714058999,...,https://ipfs.io/ipfs/QmNf1UsmdGaMbpatQ6toXSkzD...,https://ipfs.io/ipfs/bafybeibc5sgo2plmjkq2tzmh...,https://opensea.io/assets/ethereum/0xbd3531da5...,2024-04-25T15:30:03.434971,False,False,11400000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,WETH


Index(['event_type', 'order_hash', 'chain', 'protocol_address', 'closing_date',
       'quantity', 'seller', 'buyer', 'transaction', 'event_timestamp',
       'nft.identifier', 'nft.collection', 'nft.contract',
       'nft.token_standard', 'asset.name', 'nft.description', 'nft.image_url',
       'nft.metadata_url', 'nft.opensea_url', 'nft.updated_at',
       'nft.is_disabled', 'nft.is_nsfw', 'payment.quantity',
       'payment.token_address', 'payment.decimals', 'payment.symbol'],
      dtype='object')

50

### Individual NFT Info

#### query parameters from https://docs.opensea.io/reference/get_nft

In [152]:
# PATH PARAMETERS

# note: nft contract is the same for the whole collection

# identify nft contract (public blockchain address)
# this creates an array
protocol_address = events_df["nft.contract"].unique()

# eliminate the null by getting the max of the array
protocol_address = np.max(protocol_address)

# find unique values for chain (currency)
chain = events_df["chain"].unique().tolist()

# write value without list
chain = chain[0]

# token ID in a list for iteration
token_id_list = events_df["nft.identifier"].to_list()

In [154]:
# parsing features function

def get_features (nft):
    features = {}
    features["Background"] = nft["traits"][0]["value"]
    features["Skin"] = nft["traits"][1]["value"]
    features["Face"] = nft["traits"][2]["value"]
    features["Head"] = nft["traits"][3]["value"]
    features["Body"] = nft["traits"][4]["value"]
    features["Rarity"] = nft["rarity"]["rank"]
    
    return features

In [156]:
nfts_dict = {}
token_id = token_id_list[0]

for token_id in token_id_list:
    url = f"https://api.opensea.io/api/v2/chain/{chain}/contract/{protocol_address}/nfts/{token_id}"
    headers = {
        "accept": "application/json",
        "x-api-key": "5b828d394dd74a99ad2f9142157215d2"
    }

    response = requests.get(url, headers=headers)

    # Formatting as json
    nfts_json = response.json()
    
    nfts_dict[token_id] = get_features(nfts_json["nft"])

In [161]:
nfts_dict

nfts_df = pd.DataFrame(nfts_dict).transpose()

nfts_df

,Background,Skin,Face,Head,Body,Rarity
4549,Baby Pink,Blushing,Purple,Turtleneck Blue,Bowl Cut,3351
2689,Baby Pink,Purple,Sombrero,Cross Eyed,Lab Coat,3913
5490,Maroon,Scarf Blue,Tangerine,Cucumbers,Hat Red,3155
4251,Beige,Normal,Tank Top Yellow,Flat Cap Blue,Cream,6296
5968,Normal,Cute,Backwards Hat Red,Purple,Surfboard Necklace,8045
563,Mint,Dark Gray,Huddle Shirt,Blushing,Afro With Pick,8099
4436,Light Gray,Purple,Bow Tie Black,Hat Red,Handlebar Bear,4176
3114,Light Gray,Beard,Purple,Turtleneck Green,Headband,6306
8390,Blue,Santa Hat,Normal,Cute,Lei Purple,8016
5996,Normal,Normal,Pink,Scarf Green,Camo Helmet,8098
